# Struts

Subtitle

## Setup

### Imports & Env. Settings

In [22]:
# Imports
import handcalcs # https://github.com/connorferster/handcalcs/wiki
from handcalcs import render # https://github.com/connorferster/handcalcs?tab=readme-ov-file#basic-usage-1-as-a-jupyter-cell-magic-render
import forallpeople as si # https://connorferster.github.io/forallpeople/basic_usage.html
from modules.section import UniversalColumn
from modules.strength import design_strength

In [23]:
# Environment config.
# --> SI units structural env ( https://connorferster.github.io/forallpeople/environments.html#Environments )
si.environment("structural", top_level=True) # load SI units for structural eng. into the environment, top_level=True defines them in the top-level namespace (as vars)
# -> Handcalcs environment settings for TeX rendering
handcalcs.set_option("latex_block_start", "$")
handcalcs.set_option("latex_block_end", "$")
handcalcs.set_option("math_environment_start", "aligned")
handcalcs.set_option("math_environment_end", "aligned")

### Question Data

In [24]:
%%render params 2 # Loads/Moments, lengths, qn. params
# Actions
F_c = 1400 * kN

# Dimensions
L = 6 * m

# Strengths
p_y = 275 * MPa

<IPython.core.display.Latex object>

### Trial Section

In [25]:
trial_section = UniversalColumn.create(designation="203x203x60")
# ... UB not implemented, should be in rework of sections module
# iff st to check ASCII x & replace, + trimming

In [26]:
%%render params 1 # BS Notation
D = trial_section.D * mm
B = trial_section.B * mm
d = trial_section.d * mm
t = trial_section.t * mm
T = trial_section.T * mm

S_xx = trial_section.S_xx * 1000*mm**3
S_yy = trial_section.S_yy * 1000*mm**3
Z_xx = trial_section.Z_xx * 1000*mm**3
Z_yy = trial_section.Z_yy * 1000*mm**3
r_xx = trial_section.r_xx * 10*mm
r_yy = trial_section.r_yy * 10*mm
A = trial_section.A * 100*mm**2

<IPython.core.display.Latex object>

In [27]:
%%render params 1 # EC3 Notation
h = D
b = B
t_w = t
t_f = T

W_ply = S_xx
W_plz = S_yy
W_ely = Z_xx
W_elz = Z_yy
i_yy = r_xx
i_zz = r_yy
A = trial_section.A * 100*mm**2

<IPython.core.display.Latex object>

<!-- ### Classification -->

### Effective Length $L_{\text{E}}$ / $L_{\text{Cr}}$

Table 22 BS 5950
_ENV 1993-1-1:2004_

In [28]:
%%render short 2 # Uncomment correct value, defaults to 1.0 L
# L_E = 0.7 * L
# L_E = 0.85 * L
L_E = 1.0 * L
# L_E = 1.2 * L
# L_E = 1.5 * L
# L_E = 2.0 * L

# L_Cr = L_E

<IPython.core.display.Latex object>

## BS 5950

### Classification

In [29]:
# Strength class (Defaults to S275) Table 9 BS
thickest = max(trial_section.t, trial_section.T)
p_y = design_strength(thickness=thickest, steel_grade=275)
print(f"Thickness: {thickest} mm")
print(f"Design strength: {p_y} MPa")

Thickness: 14.2 mm
Design strength: 275 MPa


In [30]:
# Section Class Table 11 BS

### Slenderness Ratio $\lambda$ 

In [31]:
%%render 2 # Slenderness ratio, Cl 4.7.2
lamb = L_E / r_yy

<IPython.core.display.Latex object>

### Slenderness check

In [32]:
%%render params 2 # b/T and d/t
bT = trial_section.b_T # b/T
dt = trial_section.d_t # d/t

<IPython.core.display.Latex object>

In [33]:
# %%render short 3 # too much work and irrelevant
# epsilon = (275 / p_y)**0.5

# if bT < 15*epsilon: SLENDERWEB = False
# if dt < 40*epsilon: SLENDERFLANGE = False

### $p_{\text{c}}$

Read off or interpolate values of 

In [34]:
%%render params 2 # compressive strength pc, Table 24
# %%render params short 2 # ... checking rendering error
p_c = 208.0 * MPa

<IPython.core.display.Latex object>

### $P_{\text{c}}$

In [35]:
%%render 3 # Compression resistance, Cl 4.7.4, and Fc < Pc
P_c = A * p_c

if F_c < P_c: CompressionCapacity = "OK✅"
elif F_c >= P_c: CompressionCapacity = "INADEQUATE🚫"

<IPython.core.display.Latex object>

## Eurocode 3

### Equivalent params

In [36]:
%%render params 2
N_Ed = F_c
L_Cr = L_E
f_y = p_y * MPa

E = 210 * GPa
gamma_m_0 = 1.0
gamma_m_1 = 1.0
pi = 3.14159265359

<IPython.core.display.Latex object>

### $N_{\text{c,Rd}}$

In [37]:
%%render short 2 # Cross ... Cl.
N_cRd = A * f_y / gamma_m_0

<IPython.core.display.Latex object>

### $\lambda_{\text{1}}$ and $\lambda_{\text{z}}$

In [38]:
%%render short 3 # Slenderness
L_Cr
lamb_1 = pi * (E / f_y)**0.5

<IPython.core.display.Latex object>

In [39]:
%%render short 3 # Relative Slenderness
lamb_z = (L_Cr/i_zz) * (1/lamb_1)
# lamb_z = L_Cr/i_zz * 1/lamb_1

<IPython.core.display.Latex object>

### Buckling Curve

In [40]:
%%render short 3 # Select buckling curve ...
hb = h / b # h/b
t_f

<IPython.core.display.Latex object>

### $\Phi$ and $\chi$

In [41]:
%%render short 3 # having selected alpha from ...
alpha = 0.49
Phi = 0.5 * (1 + alpha*(lamb_z -0.2) + lamb_z**2)
chi = 1 / (Phi + (Phi**2 - lamb_z**2)**0.5)

<IPython.core.display.Latex object>

### $N_{\text{b,Rd}}$

In [42]:
%%render short 2 ##
N_bRd = chi * A * f_y / gamma_m_1

if N_Ed < N_bRd: CompressionResistance = "OK✅"
elif N_Ed >= N_bRd: CompressionResistance = "INADEQUATE🚫"

<IPython.core.display.Latex object>